In [1]:
import pandas as pd
import numpy as np

def fibonacci(n): 
    fn = [0, 1,]
    for i in range(2, n+1):
        fn.append(fn[i-1] + fn[i-2])
    return fn


N = np.arange(16)
data = {'n': N, 'Fibonacci(n)': fibonacci(15)}
df = pd.DataFrame(data)

In [2]:
def fib_search(f, a, b, n):
    F = fibonacci(n) 
    L0 = b - a # Initial interval of uncertainty
    R1 = L0
    Li = (F[n-2]/F[n])*L0 
    
    R = [Li/L0]

    for i in range(2, n+1):
        if Li > L0/2:
            x1 = b - Li
            x2 = a + Li
        else:
            x1 = a + Li
            x2 = b - Li
            
        f1, f2 = f(x1), f(x2)
        
        if f1 < f2:
            b = x2
            Li = (F[n - i]/F[n - (i - 2)])*L0 # New interval of uncertainty
        elif f1 > f2:
            a = x1
            Li = (F[n - i]/F[n - (i - 2)])*L0 # New interval of uncertainty
        else:
            a, b = x1, x2
            Li = (F[n - i]/F[n - (i - 2)])*(b - a) # New interval of uncertainty
            
        L0 = b - a
        #R += [Li/R1,] 
        
    if f1 <= f2:
        return [x1, f(x1)] 
    else:
        return [x2, f(x2)]

In [3]:
def f(x):
    return x**2

interval = fib_search(f,-5,15,7)
min = (interval[0]+interval[1])/2
print(f"[{interval[0]}, {interval[1]}]")
print(min)

[-0.3846153846153839, 0.1479289940828397]
-0.11834319526627211


In [4]:
import math

def golden_section_search(f, a, b, tol=1e-5, max_iter=100):
    r = (math.sqrt(5) - 1) / 2  # golden ratio 
    
    x1 = b - r * (b - a)
    x2 = a + r * (b - a)
    f1 = f(x1)
    f2 = f(x2)

    iter_count = 0
    while abs(b - a) > tol :#and iter_count < max_iter:
        if f1 < f2:
            b = x2
            x2 = x1
            f2 = f1
            x1 = b - r * (b - a)
            f1 = f(x1)
        else:
            a = x1
            x1 = x2
            f1 = f2
            x2 = a + r * (b - a)
            f2 = f(x2)
        iter_count += 1

    x_min = (a + b) / 2
    return x_min, f(x_min), iter_count


In [5]:
x_min, f_min, iters = golden_section_search(f, a=0, b=5)
print(f"Minimum at x = {x_min}, f(x) = {f_min}, iterations = {iters}")

Minimum at x = 3.5179209931301263e-06, f(x) = 1.2375768113905654e-11, iterations = 28


In [10]:
!pip install sympy


   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ----------------------------- ---------- 4.7/6.3 MB 35.9 MB/s eta 0:00:01
   ---------------------------------------- 6.3/6.3 MB 32.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
   --------------------------------------- 536.2/536.2 kB 18.4 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [mpmath]
   ---------------------------------------- 0/2 [mpmath]
   -------------------- ------------------- 1/2 [sympy]
   -------------------- ------------------- 1/2 [sympy]
   -------------------- ------------------- 1/2 [sympy]
   -------------------- ------------------- 1/2 [sympy]
   -------------------- ------------------- 1/2 [sympy]
   -------------------- ------------------- 1/2 [sympy]
   -------------------- ------------------- 1/2 [sympy]
   -------------------- ------------------- 1/2 [sympy]
   -------------------- ------------------- 1/2 [sympy]
   ----

In [6]:
from scipy.optimize import minimize_scalar
result = minimize_scalar(f, method='golden', tol=10**-5)
print(result)

 message: 
          Optimization terminated successfully;
          The returned value satisfies the termination criteria
          (using xtol = 1e-05 )
 success: True
     fun: 0.0
       x: 1.57172245561641e-162
     nit: 799
    nfev: 804


### Steepest Descent method

In [7]:
from sympy import symbols, Poly, Matrix, zeros, lambdify,diff

In [8]:
def steepest_descent(f,x_vals):
    n=len(x_vals)
    x_syms= symbols(f'x1:{n+1}')

    poly = Poly(f, *x_syms)
    terms = poly.terms()

    Q = zeros(n, n)
    B = zeros(n, 1)

    for powers, coeff in terms:
        nonzero_indices = [i for i, p in enumerate(powers) if p != 0] #we donot take costant because if diffrentiated it becomes 0

        if len(nonzero_indices) == 2:
            i, j = nonzero_indices
            Q[i, j] += coeff
            Q[j, i] += coeff  # ensure symmetry

        elif len(nonzero_indices) == 1:
            i = nonzero_indices[0]
            if powers[i] == 2:
                Q[i, i] += 2 * coeff
            elif powers[i] == 1:
                B[i, 0] += coeff

    
    x_vec = Matrix(x_vals).reshape(n, 1)
    while True:
        grad = Q * x_vec + B

        if grad ==  zeros(n,1):
            break  # Gradient is zero → converged

        numerator = (grad.T * grad)[0]
        denominator = (grad.T * Q * grad)[0]

        # if denominator == 0:
        #     break

        alpha = numerator / denominator
        x_new = x_vec - alpha * grad
        x_vec = x_new
        
    return x_vec

In [9]:
x1, x2 = symbols('x1 x2 ')
f = x1**2 + 2*x2 + x2**2 + 4

result = steepest_descent(f, [2,1])
print("Minimum at:", result)

Minimum at: Matrix([[0], [-1]])


### Newton's Method

In [10]:
import numpy as np



In [11]:
def newton_method(f,x_vals,tol=1e-6, max_iter=50):
    n = len(x_vals)
    x_syms = symbols(f'x1:{n+1}')

    poly = Poly(f,*x_syms)
    terms = poly.terms()
    
    # Gradient (vector of first partials)
    grad_f = Matrix([diff(f, var) for var in x_syms])
                             
    # Hessian (matrix of second partials)
    H_f = Matrix(n, n, lambda i, j: diff(f, x_syms[i], x_syms[j]))

    
    grad_func = lambdify(x_syms, grad_f, modules='numpy')
    hess_func = lambdify(x_syms, H_f, modules='numpy')

    x = np.array(x_vals, dtype=np.float64)

    while True:
        grad_val = np.array(grad_func(*x), dtype=np.float64).reshape(n, 1)
        hess_val = np.array(hess_func(*x), dtype=np.float64)

        # Convergence check
        if np.linalg.norm(grad_val) < tol:
            break

        try:
            delta = np.linalg.solve(hess_val, grad_val)
        except np.linalg.LinAlgError:
            print("Hessian is singular or ill-conditioned. Stopping.")
            break

        x = x - delta.flatten()

    return x

    

In [12]:
x1, x2 = symbols('x1 x2')
f = 8*x1**2 - 4*x1*x2 + 5*x2**2 

result = newton_method(f, [5, 2])
print("Minimum at:", result)

Minimum at: [0. 0.]


In [13]:
x1, x2, x3 = symbols('x1 x2 x3')
f = x1**4 + x1**3 - x1 + x2**4 - x2**2 + x2 + x3**2 - x3 + x1*x2*x3

result = newton_method(f, [1, -1, 1])
print("Minimum at:", result)

Minimum at: [ 0.57085597 -0.93955591  0.76817555]
